In [1]:
import requests 
import pandas as pd 
from datetime import date
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
import time 
import json
import numpy as np
import plotly as pt 

In [14]:
todays_date=date.today()
current_year=todays_date.year
years=list(range(1990,current_year))


Web scrapping （Request and Selenium)

In [ ]:
init_url="https://www.basketball-reference.com/awards/awards_{}.html"

In [ ]:
# Web scrapping: MPV awards voting stats from 1990 to 2021
for year in years:
    url=init_url.format(year)
    data_yearly_awards=requests.get(url)

    with open("yearly_awards/awards_{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(data_yearly_awards.text)


In [ ]:
# HTML parsing from downloaded HTML files 
list_dfs=[]
for year in years:
    with open("yearly_awards/awards_{}.html".format(year),encoding="utf-8") as f:
        page=f.read()
    soup=bs(page,"html.parser")
    soup.find("tr", class_="over_header").decompose() #remove the extra header by using decompose() method 
    mvp_table=soup.find(id="mvp")#find the mvp voting table by using global unique id 
    mvp_df=pd.read_html(str(mvp_table))[0] # let panda read html by converting html to string 
    mvp_df['Year']=year
    list_dfs.append(mvp_df)



In [ ]:
index=list(range(0,len(years)))
zipped=list(zip(years,index))
index_dict={year:index for (year,index) in zipped}


In [ ]:
mvp_voting_table=pd.concat(list_dfs) # concat list of dataframes
mvp_voting_table

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Magic Johnson,30,LAL,27.0,636.0,920,0.691,79,37.2,...,6.6,11.5,1.7,0.4,0.480,0.384,0.890,16.5,0.270,1990
1,2,Charles Barkley,26,PHI,38.0,614.0,920,0.667,79,39.1,...,11.5,3.9,1.9,0.6,0.600,0.217,0.749,17.3,0.269,1990
2,3,Michael Jordan,26,CHI,21.0,564.0,920,0.613,82,39.0,...,6.9,6.3,2.8,0.7,0.526,0.376,0.848,19.0,0.285,1990
3,4,Karl Malone,26,UTA,2.0,214.0,920,0.233,82,38.1,...,11.1,2.8,1.5,0.6,0.562,0.372,0.762,15.9,0.245,1990
4,5,Patrick Ewing,27,NYK,1.0,162.0,920,0.176,82,38.6,...,10.9,2.2,1.0,4.0,0.551,0.250,0.775,13.5,0.205,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,11,Russell Westbrook,32,WAS,0.0,5.0,1010,0.005,65,36.4,...,11.5,11.7,1.4,0.4,0.439,0.315,0.656,3.7,0.075,2021
11,12,Ben Simmons,24,PHI,0.0,3.0,1010,0.003,58,32.4,...,7.2,6.9,1.6,0.6,0.557,0.300,0.613,6.0,0.153,2021
12,13T,James Harden,31,TOT,0.0,1.0,1010,0.001,44,36.6,...,7.9,10.8,1.2,0.8,0.466,0.362,0.861,7.0,0.208,2021
13,13T,LeBron James,36,LAL,0.0,1.0,1010,0.001,45,33.4,...,7.7,7.8,1.1,0.6,0.513,0.365,0.698,5.6,0.179,2021


In [ ]:
mvp_voting_table[mvp_voting_table["Year"]==2021].iloc[0]

Rank                  1
Player     Nikola Jokić
Age                  25
Tm                  DEN
First              91.0
Pts Won           971.0
Pts Max            1010
Share             0.961
G                    72
MP                 34.6
PTS                26.4
TRB                10.8
AST                 8.3
STL                 1.3
BLK                 0.7
FG%               0.566
3P%               0.388
FT%               0.868
WS                 15.6
WS/48             0.301
Year               2021
Name: 0, dtype: object

In [ ]:
mvp_voting_table.to_csv("mvp_votings.csv")

In [ ]:
#init Selenium driver to scrap per game statistics 
driver= webdriver.Chrome(executable_path="chromedriver")

In [ ]:
pergame_stats_url="https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
for year in years:
    url=pergame_stats_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)

    html=driver.page_source
    with open("per_game_stats/stats_{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(html)

In [18]:
list_dfs=[]
for year in years:
    with open("per_game_stats/stats_{}.html".format(year),encoding="utf-8") as f:
        page = f.read()
    
    soup=bs(page,"html.parser")
    soup.find("tr", class_="thead").decompose() #remove the extra header by using decompose() method 
    player_table=soup.find(id="per_game_stats")#find the mvp voting table by using global unique id 
    player_stats_df=pd.read_html(str(player_table))[0] # let panda read html by converting html to string 
    player_stats_df['Year']=year
    list_dfs.append(player_stats_df)

In [19]:
player_per_game_stats=pd.concat(list_dfs)

In [28]:
player_per_game_stats.loc[player_per_game_stats.Player=="Stephen Curry",]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
123,100,Stephen Curry,PG,21,GSW,80,77,36.2,6.6,14.3,...,0.6,3.9,4.5,5.9,1.9,0.2,3.0,3.2,17.5,2010
157,102,Stephen Curry,PG,22,GSW,74,74,33.6,6.8,14.2,...,0.7,3.2,3.9,5.8,1.5,0.3,3.1,3.1,18.6,2011
113,102,Stephen Curry,PG,23,GSW,26,23,28.2,5.6,11.4,...,0.6,2.8,3.4,5.3,1.5,0.3,2.5,2.4,14.7,2012
122,97,Stephen Curry,PG,24,GSW,78,78,38.2,8.0,17.8,...,0.8,3.3,4.0,6.9,1.6,0.2,3.1,2.5,22.9,2013
153,111,Stephen Curry,PG,25,GSW,78,78,36.5,8.4,17.7,...,0.6,3.7,4.3,8.5,1.6,0.2,3.8,2.5,24.0,2014
139,112,Stephen Curry,PG,26,GSW,80,80,32.7,8.2,16.8,...,0.7,3.6,4.3,7.7,2.0,0.2,3.1,2.0,23.8,2015
120,105,Stephen Curry,PG,27,GSW,79,79,34.2,10.2,20.2,...,0.9,4.6,5.4,6.7,2.1,0.2,3.3,2.0,30.1,2016
123,98,Stephen Curry,PG,28,GSW,79,79,33.4,8.5,18.3,...,0.8,3.7,4.5,6.6,1.8,0.2,3.0,2.3,25.3,2017
150,120,Stephen Curry,PG,29,GSW,51,51,32.0,8.4,16.9,...,0.7,4.4,5.1,6.1,1.6,0.2,3.0,2.2,26.4,2018
164,124,Stephen Curry,PG,30,GSW,69,69,33.8,9.2,19.4,...,0.7,4.7,5.3,5.2,1.3,0.4,2.8,2.4,27.3,2019


In [29]:
player_per_game_stats.to_csv("player_per_game_stats.csv")

In [12]:
team_standing="https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [21]:
for year in years:
    team_standing_url=team_standing.format(year)

    yearly_team_standing=requests.get(team_standing_url)

    with open("team_standing/standing_{}.html".format(year),"w+",encoding="utf-8") as f:
        f.write(yearly_team_standing.text)


In [22]:
dfs=[]
for year in years:
    with open("team_standing/standing_{}.html".format(year),encoding="utf-8") as f:
        page=f.read()

    soup=bs(page,"html.parser")
    soup.find("tr", class_="thead").decompose() #remove the extra header by using decompose() method 
    team_standing_table=soup.find(id="divs_standings_E")
    team_standing_df=pd.read_html(str(team_standing_table))[0] # let panda read html by converting html to string 
    team_standing_df['Year']=year
    team_standing_df['Team Name']=team_standing_df['Eastern Conference']
    dfs.append(team_standing_df)


    soup=bs(page,"html.parser")
    soup.find("tr", class_="thead").decompose() #remove the extra header by using decompose() method 
    team_standing_table=soup.find(id="divs_standings_W")
    team_standing_df=pd.read_html(str(team_standing_table))[0] # let panda read html by converting html to string 
    team_standing_df['Year']=year
    team_standing_df['Team Name']=team_standing_df['Western Conference']
    dfs.append(team_standing_df)

In [23]:
team_standing_df=pd.concat(dfs)

In [26]:
team_standing_df

,Eastern Conference,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team Name,Western Conference
0,Philadelphia 76ers*,53,29,.646,—,110.2,105.2,4.23,1990,Philadelphia 76ers*,NaN
1,Boston Celtics*,52,30,.634,1.0,110.0,106.0,3.23,1990,Boston Celtics*,NaN
2,New York Knicks*,45,37,.549,8.0,108.3,106.9,0.78,1990,New York Knicks*,NaN
3,Washington Bullets,31,51,.378,22.0,107.7,109.9,-2.43,1990,Washington Bullets,NaN
4,Miami Heat,18,64,.220,35.0,100.6,110.3,-9.59,1990,Miami Heat,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13,NaN,42,30,.583,—,112.4,110.2,2.26,2021,Dallas Mavericks*,Dallas Mavericks*
14,NaN,38,34,.528,4.0,113.3,112.3,1.07,2021,Memphis Grizzlies*,Memphis Grizzlies*
15,NaN,33,39,.458,9.0,111.1,112.8,-1.58,2021,San Antonio Spurs,San Antonio Spurs
16,NaN,31,41,.431,11.0,114.6,114.9,-0.20,2021,New Orleans Pelicans,New Orleans Pelicans


In [27]:
team_standing_df=team_standing_df.drop(['Eastern Conference','Western Conference'],axis=1 )

In [34]:
team_standing_df.loc[team_standing_df['Year']==2021,]

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team Name
0,49,23,.681,—,113.6,108.1,5.28,2021,Philadelphia 76ers*
1,48,24,.667,1.0,118.6,114.1,4.24,2021,Brooklyn Nets*
2,41,31,.569,8.0,107.0,104.7,2.13,2021,New York Knicks*
3,36,36,.500,13.0,112.6,111.2,1.32,2021,Boston Celtics*
4,27,45,.375,22.0,111.3,111.7,-0.54,2021,Toronto Raptors
5,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,2021,Central Division
6,46,26,.639,—,120.1,114.2,5.57,2021,Milwaukee Bucks*
7,34,38,.472,12.0,115.3,115.3,-0.13,2021,Indiana Pacers
8,31,41,.431,15.0,110.7,111.6,-0.94,2021,Chicago Bulls
9,22,50,.306,24.0,103.8,112.3,-8.19,2021,Cleveland Cavaliers


In [35]:
team_standing_df.to_csv("team_standing.csv")

Read local csv file 

In [8]:
mvp_votings=pd.read_csv("mvp_votings.csv")
mvp_votings=mvp_votings.drop("Unnamed: 0",axis=1)
mvp_votings

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Magic Johnson,30,LAL,27.0,636.0,920,0.691,79,37.2,...,6.6,11.5,1.7,0.4,0.480,0.384,0.890,16.5,0.270,1990
1,2,Charles Barkley,26,PHI,38.0,614.0,920,0.667,79,39.1,...,11.5,3.9,1.9,0.6,0.600,0.217,0.749,17.3,0.269,1990
2,3,Michael Jordan,26,CHI,21.0,564.0,920,0.613,82,39.0,...,6.9,6.3,2.8,0.7,0.526,0.376,0.848,19.0,0.285,1990
3,4,Karl Malone,26,UTA,2.0,214.0,920,0.233,82,38.1,...,11.1,2.8,1.5,0.6,0.562,0.372,0.762,15.9,0.245,1990
4,5,Patrick Ewing,27,NYK,1.0,162.0,920,0.176,82,38.6,...,10.9,2.2,1.0,4.0,0.551,0.250,0.775,13.5,0.205,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,11,Russell Westbrook,32,WAS,0.0,5.0,1010,0.005,65,36.4,...,11.5,11.7,1.4,0.4,0.439,0.315,0.656,3.7,0.075,2021
484,12,Ben Simmons,24,PHI,0.0,3.0,1010,0.003,58,32.4,...,7.2,6.9,1.6,0.6,0.557,0.300,0.613,6.0,0.153,2021
485,13T,James Harden,31,TOT,0.0,1.0,1010,0.001,44,36.6,...,7.9,10.8,1.2,0.8,0.466,0.362,0.861,7.0,0.208,2021
486,13T,LeBron James,36,LAL,0.0,1.0,1010,0.001,45,33.4,...,7.7,7.8,1.1,0.6,0.513,0.365,0.698,5.6,0.179,2021


In [9]:
mvp_votings.columns

Index(['Rank', 'Player', 'Age', 'Tm', 'First', 'Pts Won', 'Pts Max', 'Share',
       'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS',
       'WS/48', 'Year'],
      dtype='object')

In [6]:
player_per_game_stats=pd.read_csv("player_per_game_stats.csv")
player_per_game_stats=player_per_game_stats.drop("Unnamed: 0",axis=1)
player_per_game_stats

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Mark Acres,C,27,ORL,80,50,21.1,1.7,3.6,...,1.9,3.5,5.4,0.8,0.5,0.3,0.9,3.1,4.5,1990
1,2,Michael Adams,PG,27,DEN,79,74,34.1,5.0,12.5,...,0.6,2.2,2.8,6.3,1.5,0.0,1.8,1.7,15.5,1990
2,3,Mark Aguirre,SF,30,DET,78,40,25.7,5.6,11.5,...,1.5,2.4,3.9,1.9,0.4,0.2,1.6,2.6,14.1,1990
3,4,Danny Ainge,PG,30,SAC,75,68,36.4,6.7,15.4,...,0.9,3.4,4.3,6.0,1.5,0.2,2.5,3.2,17.9,1990
4,5,Mark Alarie,PF,26,WSB,82,10,23.1,4.5,9.6,...,1.8,2.7,4.6,1.7,0.7,0.5,1.2,2.7,10.5,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18516,536,Delon Wright,PG,28,SAC,27,8,25.8,3.9,8.3,...,1.0,2.9,3.9,3.6,1.6,0.4,1.3,1.1,10.0,2021
18517,537,Thaddeus Young,PF,32,CHI,68,23,24.3,5.4,9.7,...,2.5,3.8,6.2,4.3,1.1,0.6,2.0,2.2,12.1,2021
18518,538,Trae Young,PG,22,ATL,63,63,33.7,7.7,17.7,...,0.6,3.3,3.9,9.4,0.8,0.2,4.1,1.8,25.3,2021
18519,539,Cody Zeller,C,28,CHO,48,21,20.9,3.8,6.8,...,2.5,4.4,6.8,1.8,0.6,0.4,1.1,2.5,9.4,2021


In [10]:
player_per_game_stats.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year'],
      dtype='object')

In [36]:
team_standing=pd.read_csv("team_standing.csv")
team_standing

,Unnamed: 0,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team Name
0,0,53,29,.646,—,110.2,105.2,4.23,1990,Philadelphia 76ers*
1,1,52,30,.634,1.0,110.0,106.0,3.23,1990,Boston Celtics*
2,2,45,37,.549,8.0,108.3,106.9,0.78,1990,New York Knicks*
3,3,31,51,.378,22.0,107.7,109.9,-2.43,1990,Washington Bullets
4,4,18,64,.220,35.0,100.6,110.3,-9.59,1990,Miami Heat
...,...,...,...,...,...,...,...,...,...,...
1058,13,42,30,.583,—,112.4,110.2,2.26,2021,Dallas Mavericks*
1059,14,38,34,.528,4.0,113.3,112.3,1.07,2021,Memphis Grizzlies*
1060,15,33,39,.458,9.0,111.1,112.8,-1.58,2021,San Antonio Spurs
1061,16,31,41,.431,11.0,114.6,114.9,-0.20,2021,New Orleans Pelicans


Connect to Azure database 

In [12]:
with open('../SQLcredentials.txt') as f:
    data = json.loads(f.read())

username=data['username']
password=data['password']

In [67]:
#Connect to Azure SQL database
import pyodbc

server = "tcp:firstserver.database.windows.net,1433"
database = "NBAprediction"
username = username
password = password
driver= '{ODBC Driver 18 for SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Connection Timeout=30')
cursor = cnxn.cursor()

In [68]:
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	May 12 2022 23:11:24 
	Copyright (C) 2022 Microsoft Corporation

